# Adapter (with caching)

> Using a cache to store the adapter's temporary objects.

We will adapt the code from the previous lesson and add a cache to our adapter that will store the adaptations that we create so that we can reuse them later and not generate them every single time we call the adapter.

First, let's brinng in the code that doesn't need changing.

In [1]:
# The API we're forced to work with
class Point:
    def __init__(self, x, y):
        self.y = y
        self.x = x

def draw_point(p):
    print('.', end='')

In [2]:
# Our application code
class Line:
    def __init__(self, start, end):
        self.end = end
        self.start = start
        
class Rectangle(list):
    """ Represented as a list of lines. """
    def __init__(self, x, y, width, height):
        super().__init__()
        self.append(Line(Point(x, y), Point(x + width, y)))
        self.append(Line(Point(x + width, y), Point(x + width, y + height)))
        self.append(Line(Point(x, y), Point(x, y + height)))
        self.append(Line(Point(x, y + height), Point(x + width, y + height)))

Now we will create a new adapter with some changes:

In [4]:
class LineToPointAdapter: # 1
    count = 0 # 7
    cache = {} # 2

    def __init__(self, line):
        self.h = hash(line) # 3
        if self.h in self.cache:
            return

        super().__init__()
        self.count += 1
        print(f'{self.count}: Generating points for line ' +
              f'[{line.start.x},{line.start.y}]→[{line.end.x},{line.end.y}]')

        left = min(line.start.x, line.end.x)
        right = max(line.start.x, line.end.x)
        top = min(line.start.y, line.end.y)
        bottom = min(line.start.y, line.end.y)

        points = [] # 4

        if right - left == 0:
            for y in range(top, bottom):
                points.append(Point(left, y))
        elif line.end.y - line.start.y == 0:
            for x in range(left, right):
                points.append(Point(x, top))

        self.cache[self.h] = points # 5

    def __iter__(self): # 6
        return iter(self.cache[self.h])

1. We don't need to inherit from `list` anymore because we will be storing the lists in a static cache, so we remove the dependency.
2. Our cache will just be a dictionary that stores the objects (lists in our case)
3. for our dictionary keys, we will be creating unique values with the `hash` function for each line. If the hash value is already in the dictionary, we simply return because we've already adapter the line. We store the hash value in the adapter instance as well.
4. Since our adapter instance isn't a list anymore, we will need a list to store the points that we will create from our line.
5. Once we've generated the list of points, we store it in the cache with the hash value as its key.
6. Because our adapter isn't a list anymore, it isn't an iterable anymore either. We implement the `__iter__` method for the class and we make it iterate over the cached line represented by the hash value stored in the adapter's instance.
7. `count` is no longer relevant but I haven't removed it from the code in order to make it easier to compare with the previous lesson's code.

Let's see our adapter in action:

In [5]:
def draw(rcs):
    print("\n\n--- Drawing some stuff ---\n")
    for rc in rcs:
        for line in rc:
            adapter = LineToPointAdapter(line)
            for p in adapter:
                draw_point(p)
                
rs = [
    Rectangle(1, 1, 10, 10),
    Rectangle(3, 3, 6, 6)
]
draw(rs)
draw(rs)



--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........1: Generating points for line [11,1]→[11,11]
1: Generating points for line [1,1]→[1,11]
1: Generating points for line [1,11]→[11,11]
..........1: Generating points for line [3,3]→[9,3]
......1: Generating points for line [9,3]→[9,9]
1: Generating points for line [3,3]→[3,9]
1: Generating points for line [3,9]→[9,9]
......

--- Drawing some stuff ---

................................

As you can see in the output, the `__init__` prints are not being generated the second time we draw `rs` because we already have the adapted lines in our cache, thus making this implementation much more efficient.